<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-sample-holdings-from-ETF" data-toc-modified-id="Import-sample-holdings-from-ETF-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import sample holdings from ETF</a></span></li><li><span><a href="#Create-ETF-Internals-instance-and-get-query-dates" data-toc-modified-id="Create-ETF-Internals-instance-and-get-query-dates-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create ETF Internals instance and get query dates</a></span></li><li><span><a href="#Download-stock-data-and-compute-ETF-internal-analytics" data-toc-modified-id="Download-stock-data-and-compute-ETF-internal-analytics-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download stock data and compute ETF internal analytics</a></span></li><li><span><a href="#Extra-code-to-run-for-single-ETF-symbol's-components" data-toc-modified-id="Extra-code-to-run-for-single-ETF-symbol's-components-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extra code to run for single ETF symbol's components</a></span></li></ul></div>

In [1]:
%load_ext watermark
%watermark

%load_ext autoreload
%autoreload 2
# import standard libs
from IPython.display import display
from IPython.core.debugger import set_trace as bp
from pathlib import PurePath, Path
import sys
import time
from collections import OrderedDict as od
import re
import os
import json
os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

# get project dir
pp = PurePath(Path.cwd()).parts[:-1]
pdir = PurePath(*pp)
script_dir = pdir / 'src' 
viz_dir = pdir / 'report' / 'figures'
data_dir = pdir / 'data'
script_dirs = list(Path(script_dir).glob('./*/'))
for sdir in script_dirs: sys.path.append(sdir.as_posix())

import _blk_utilities_
import _blk_ETF_Internals_
utils = _blk_utilities_.utilities()

from decimal import Decimal
dplaces = Decimal('0.0001')

import pandas as pd
from pandas.tseries.offsets import *
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
sz = 11
mlt = 1.66
size=(mlt * sz, sz)
import seaborn as sns
sns.set_style('white', {"xtick.major.size": 2, "ytick.major.size": 2})
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71","#f4cae4"]
sns.set_palette(sns.color_palette(flatui,7))
from collections import OrderedDict as od
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
import missingno as msno

print()
%watermark -p pandas,pandas_datareader,dask,numpy,matplotlib,seaborn

2018-04-10T19:34:13-06:00

CPython 3.6.4
IPython 6.2.1

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.13.0-37-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit
/media/bcr/HDD/Code_Backups_Sync/data/external/prices

pandas 0.22.0
pandas_datareader 0.6.0
dask 0.17.0
numpy 1.14.0
matplotlib 2.1.2
seaborn 0.8.1


## Import sample holdings from ETF

In [2]:
infp = PurePath(data_dir/'raw'/'etf_holdings')
all_etf_symbols = _blk_ETF_Internals_.formatSymbols(infp).format_all_symbols()
all_etf_symbols

OrderedDict([('SPY', 0       AAPL
              1       MSFT
              2        XOM
              3        JNJ
              4         GE
              5       AMZN
              6         FB
              7      BRK.B
              8          T
              9        JPM
              10       WFC
              11     GOOGL
              12        PG
              13      GOOG
              14       PFE
              15        VZ
              16       CVX
              17        KO
              18        HD
              19       MRK
              20        PM
              21       BAC
              22         V
              23     CMCSA
              24       DIS
              25      INTC
              26       PEP
              27      CSCO
              28         C
              29       IBM
                     ...  
              477     TGNA
              478      NRG
              479     GRMN
              480     PBCT
              481      JWN
              482    

In [3]:
all_etf_symbols.keys()

odict_keys(['SPY', 'QQQ', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY', 'FTSE', 'DAX', 'TSX', 'ASX'])

In [4]:
all_etf_symbols['SPY'].head().values

array(['AAPL', 'MSFT', 'XOM', 'JNJ', 'GE'], dtype=object)

## Create ETF Internals instance and get query dates

In [6]:
price_dir = PurePath(pdir/'data'/'external'/'prices')
etfi = _blk_ETF_Internals_.etfInternals(price_dir)
# get query dates anchored to last 52 weeks Monday
start_query, end_query = etfi._get_query_dates()
date_format = utils._create_date_format(start_query, end_query)
print('date formatting: ', start_query, end_query, date_format)

checking /media/bcr/HDD/Code_Backups_Sync/etf_internals/data/external/prices/04-10-18...
creating directory...
directory created [complete]
date formatting:  2016-04-18 00:00:00 2018-04-13 00:00:00 [Monday, 04.18.2016 - Friday, 04.13.2018]


## Download stock data and compute ETF internal analytics

In [7]:
"""
Get all ETF symbol price data

## takes ~15 mins
"""

for ETF_symbol, symbols in all_etf_symbols.items():

    """Get Symbol data"""
    ## select source data, only working with iex for now
    etfi._get_symbol_px_data(ETF_symbol, symbols)

    """Extract Close Prices"""
    px = etfi.extract_close_prices(symbols)
    ## in case no data returned
    if px is None: continue

    ## must update symbols used in case they are missing in data
    symbols = pd.Series(px.columns)

    """Set Key parameters"""
    n_periods = 252
    N_stocks = len(px.columns)
    log_returns = etfi._calc_log_returns(px)
    cuml_returns = etfi._calc_cuml_returns(log_returns)
    start_timestamp = log_returns.ix[-n_periods:].index[0]
    end_timestamp = log_returns.ix[-n_periods:].index[-1]

    """get 52 week highs/lows df's""" 
    highs_list, N_52_highs, highs_df = etfi._52_week_highs(px, symbols)
    lows_list, N_52_lows, lows_df = etfi._52_week_lows(px, symbols)

    """Calculate up/down and below/above stats"""
    avg_pct_below_highs, median_pct_below_highs, avg_pct_above_lows, median_pct_above_lows, \
    N_stocks_off_10, N_stocks_off_20, pct_stocks_off_10, pct_stocks_off_20, \
    N_stocks_up_10, N_stocks_up_20, pct_stocks_up_10, pct_stocks_up_20 = etfi._calc_high_low_metrics(highs_df, lows_df, N_stocks)

    up, N_up, nup_pct, down, N_down, ndown_pct = etfi._up_down_last_N_periods(n_periods, log_returns)

    """Construct EMAs for dataframe"""
    spans = [252, 63, 21]
    EMAS = etfi._get_EMA(px, spans, symbols)
    n_abv_252, n_abv_63, n_abv_21, pct_abv_252, pct_abv_63, pct_abv_21 = etfi._calc_ema_stats(spans, EMAS, N_stocks)

    """Calculate thresholds"""
    pos_thresh = [0., .01, .02, .03, .04, .05, .06, .07, .1, .15, .2]
    neg_thresh = [ flt * -1. for flt in pos_thresh ]

    N_pos = etfi._calc_return_thresholds(pos_thresh, log_returns.resample('1W', how='sum'), N_stocks, pos=True)
    N_neg = etfi._calc_return_thresholds(neg_thresh, log_returns.resample('1W', how='sum'), N_stocks)
    rev_N_pos = N_pos.copy().iloc[::-1]

    """average return and volatility"""
    avg_cuml_ret, avg_std = etfi._calc_avg_log_returns_vol(log_returns, n_periods)

    """Construct output dataframe for excel"""
    new_odf = etfi._create_output_dataframe(
        n_periods, start_timestamp, end_timestamp, N_stocks, \
        N_52_highs, N_52_lows, \
        avg_pct_below_highs, median_pct_below_highs, avg_pct_above_lows, median_pct_above_lows, \
        N_stocks_off_10, N_stocks_off_20, pct_stocks_off_10, pct_stocks_off_20, \
        N_stocks_up_10, N_stocks_up_20, pct_stocks_up_10, pct_stocks_up_20,
        N_up, nup_pct, N_down, ndown_pct, \
        n_abv_252, n_abv_63, n_abv_21, pct_abv_252, pct_abv_63, pct_abv_21, \
        avg_cuml_ret, avg_std
        )
    time.sleep(1.5)
    """Construct excel format"""
    today=pd.datetime.today().strftime('%m-%d-%y')
    ## TODO: TEST THIS FP OUTPUT 
    outfp=Path(pdir/'reports'/f'{today}').mkdir(parents=True,exists_ok=True)
    etfi._create_excelwriter_format(outfp, ETF_symbol, new_odf, rev_N_pos, N_neg)    

downloading symbol: AAPL | 99.80%
2y
downloading symbol: MSFT | 99.61%
2y
downloading symbol: XOM | 99.41%
2y
downloading symbol: JNJ | 99.21%
2y
downloading symbol: GE | 99.01%
2y
downloading symbol: AMZN | 98.82%
2y
downloading symbol: FB | 98.62%
2y
downloading symbol: BRK.B | 98.42%
2y
downloading symbol: T | 98.22%
2y
downloading symbol: JPM | 98.03%
2y
downloading symbol: WFC | 97.83%
2y
downloading symbol: GOOGL | 97.63%
2y
downloading symbol: PG | 97.44%
2y
downloading symbol: GOOG | 97.24%
2y
downloading symbol: PFE | 97.04%
2y
downloading symbol: VZ | 96.84%
2y
downloading symbol: CVX | 96.65%
2y
downloading symbol: KO | 96.45%
2y
downloading symbol: HD | 96.25%
2y
downloading symbol: MRK | 96.06%
2y
downloading symbol: PM | 95.86%
2y
downloading symbol: BAC | 95.66%
2y
downloading symbol: V | 95.46%
2y
downloading symbol: CMCSA | 95.27%
2y
downloading symbol: DIS | 95.07%
2y
downloading symbol: INTC | 94.87%
2y
downloading symbol: PEP | 94.67%
2y
downloading symbol: CSCO | 9

downloading symbol: PGR | 55.23%
2y
downloading symbol: NLSN | 55.03%
2y
downloading symbol: BXP | 54.83%
2y
downloading symbol: TSN | 54.64%
2y
downloading symbol: TROW | 54.44%
2y
downloading symbol: AAL | 54.24%
2y
downloading symbol: K | 54.04%
2y
downloading symbol: MTB | 53.85%
2y
downloading symbol: WEC | 53.65%
2y
downloading symbol: DLPH | 53.45%
2y
downloading symbol: CMI | 53.25%
2y
downloading symbol: MPC | 53.06%
2y
downloading symbol: DVN | 52.86%
2y
downloading symbol: TAP | 52.66%
2y
downloading symbol: TYC | 52.47%
2y
'TYC' TYC
downloading symbol: ADI | 52.27%
2y
downloading symbol: APH | 52.07%
2y
downloading symbol: HIG | 51.87%
2y
downloading symbol: MYL | 51.68%
2y
downloading symbol: WLTW | 51.48%
2y
downloading symbol: ES | 51.28%
2y
downloading symbol: PAYX | 51.08%
2y
downloading symbol: SWK | 50.89%
2y
downloading symbol: ROP | 50.69%
2y
downloading symbol: DPS | 50.49%
2y
downloading symbol: IP | 50.30%
2y
downloading symbol: NEM | 50.10%
2y
downloading symbo

downloading symbol: LEG | 11.05%
2y
downloading symbol: AIV | 10.85%
2y
downloading symbol: QRVO | 10.65%
2y
downloading symbol: TE | 10.45%
2y
'TE' TE
downloading symbol: HP | 10.26%
2y
downloading symbol: FMC | 10.06%
2y
downloading symbol: ALLE | 9.86%
2y
downloading symbol: STX | 9.66%
2y
downloading symbol: UA.C | 9.47%
2y
'UA.C' UA.C
downloading symbol: FLS | 9.27%
2y
downloading symbol: URI | 9.07%
2y
downloading symbol: DISCK | 8.88%
2y
downloading symbol: JEC | 8.68%
2y
downloading symbol: FTI | 8.48%
2y
downloading symbol: PHM | 8.28%
2y
downloading symbol: LUK | 8.09%
2y
downloading symbol: PKI | 7.89%
2y
downloading symbol: FTR | 7.69%
2y
downloading symbol: SNI | 7.50%
2y
downloading symbol: AIZ | 7.30%
2y
downloading symbol: ZION | 7.10%
2y
downloading symbol: RL | 6.90%
2y
downloading symbol: SPLS | 6.71%
2y
'SPLS' SPLS
downloading symbol: HAR | 6.51%
2y
'HAR' HAR
downloading symbol: RHI | 6.31%
2y
downloading symbol: MUR | 6.11%
2y
downloading symbol: SWN | 5.92%
2y
dow

downloading symbol: HSIC | 48.11%
2y
downloading symbol: AAL | 47.17%
2y
downloading symbol: PCAR | 46.23%
2y
downloading symbol: VRSK | 45.28%
2y
downloading symbol: FAST | 44.34%
2y
downloading symbol: SRCL | 43.40%
2y
downloading symbol: AAPL | 42.45%
2y
downloading symbol: MSFT | 41.51%
2y
downloading symbol: GOOG | 40.57%
2y
downloading symbol: FB | 39.62%
2y
downloading symbol: GOOGL | 38.68%
2y
downloading symbol: INTC | 37.74%
2y
downloading symbol: CSCO | 36.79%
2y
downloading symbol: QCOM | 35.85%
2y
downloading symbol: TXN | 34.91%
2y
downloading symbol: BIDU | 33.96%
2y
downloading symbol: ADBE | 33.02%
2y
downloading symbol: PYPL | 32.08%
2y
downloading symbol: CTSH | 31.13%
2y
downloading symbol: ADP | 30.19%
2y
downloading symbol: AVGO | 29.25%
2y
downloading symbol: EBAY | 28.30%
2y
downloading symbol: YHOO | 27.36%
2y
'YHOO' YHOO
downloading symbol: ATVI | 26.42%
2y
downloading symbol: NXPI | 25.47%
2y
downloading symbol: INTU | 24.53%
2y
downloading symbol: EA | 23.58

downloading symbol: HCP | 40.66%
2y
downloading symbol: FITB | 39.56%
2y
downloading symbol: ESS | 38.46%
2y
downloading symbol: O | 37.36%
2y
downloading symbol: BEN | 36.26%
2y
downloading symbol: DLR | 35.16%
2y
downloading symbol: IVZ | 34.07%
2y
downloading symbol: RF | 32.97%
2y
downloading symbol: CFG | 31.87%
2y
downloading symbol: PFG | 30.77%
2y
downloading symbol: HST | 29.67%
2y
downloading symbol: KIM | 28.57%
2y
downloading symbol: EXR | 27.47%
2y
downloading symbol: L | 26.37%
2y
downloading symbol: LNC | 25.27%
2y
downloading symbol: KEY | 24.18%
2y
downloading symbol: FRT | 23.08%
2y
downloading symbol: CINF | 21.98%
2y
downloading symbol: XL | 20.88%
2y
downloading symbol: SLG | 19.78%
2y
downloading symbol: MAC | 18.68%
2y
downloading symbol: UDR | 17.58%
2y
downloading symbol: AMG | 16.48%
2y
downloading symbol: UNM | 15.38%
2y
downloading symbol: IRM | 14.29%
2y
downloading symbol: CBG | 13.19%
2y
downloading symbol: CMA | 12.09%
2y
downloading symbol: HBAN | 10.99

...

downloading symbol: .L | 99.02%
2y
'.L' .L
downloading symbol: .L | 98.04%
2y
'.L' .L
downloading symbol: .L | 97.06%
2y
'.L' .L
downloading symbol: .L | 96.08%
2y
'.L' .L
downloading symbol: .L | 95.10%
2y
'.L' .L
downloading symbol: .L | 94.12%
2y
'.L' .L
downloading symbol: .L | 93.14%
2y
'.L' .L
downloading symbol: .L | 92.16%
2y
'.L' .L
downloading symbol: .L | 91.18%
2y
'.L' .L
downloading symbol: .L | 90.20%
2y
'.L' .L
downloading symbol: .L | 89.22%
2y
'.L' .L
downloading symbol: .L | 88.24%
2y
'.L' .L
downloading symbol: .L | 87.25%
2y
'.L' .L
downloading symbol: .L | 86.27%
2y
'.L' .L
downloading symbol: .L | 85.29%
2y
'.L' .L
downloading symbol: .L | 84.31%
2y
'.L' .L
downloading symbol: .L | 83.33%
2y
'.L' .L
downloading symbol: .L | 82.35%
2y
'.L' .L
downloading symbol: .L | 81.37%
2y
'.L' .L
downloading symbol: .L | 80.39%
2y
'.L' .L
downloading symbol: .L | 79.41%
2y
'.L' .L
downloading symbol: .L | 78.43%
2y
'.L' .L
downloading symbol: .L | 77.45%
2y
'.L' .L
downloading

'BAYN.DE' BAYN.DE
downloading symbol: SIE.DE | 93.33%
2y
'SIE.DE' SIE.DE
downloading symbol: SAP.DE | 90.00%
2y
'SAP.DE' SAP.DE
downloading symbol: ALV.DE | 86.67%
2y
'ALV.DE' ALV.DE
downloading symbol: DAI.DE | 83.33%
2y
'DAI.DE' DAI.DE
downloading symbol: BAS.DE | 80.00%
2y
'BAS.DE' BAS.DE
downloading symbol: DTE.DE | 76.67%
2y
'DTE.DE' DTE.DE
downloading symbol: MUV2.DE | 73.33%
2y
'MUV2.DE' MUV2.DE
downloading symbol: BMW.DE | 70.00%
2y
'BMW.DE' BMW.DE
downloading symbol: FRE.DE | 66.67%
2y
'FRE.DE' FRE.DE
downloading symbol: DBK.DE | 63.33%
2y
'DBK.DE' DBK.DE
downloading symbol: DPW.DE | 60.00%
2y
'DPW.DE' DPW.DE
downloading symbol: CON.DE | 56.67%
2y
'CON.DE' CON.DE
downloading symbol: LIN.DE | 53.33%
2y
'LIN.DE' LIN.DE
downloading symbol: VOW3.DE | 50.00%
2y
'VOW3.DE' VOW3.DE
downloading symbol: ADS.DE | 46.67%
2y
'ADS.DE' ADS.DE
downloading symbol: EOAN.DE | 43.33%
2y
'EOAN.DE' EOAN.DE
downloading symbol: FME.DE | 40.00%
2y
'FME.DE' FME.DE
downloading symbol: HEN3.DE | 36.67%
2

'CP.TO' CP.TO
downloading symbol: T.TO | 69.49%
2y
'T.TO' T.TO
downloading symbol: MG.TO | 67.80%
2y
'MG.TO' MG.TO
downloading symbol: POT.TO | 66.10%
2y
'POT.TO' POT.TO
downloading symbol: RCI.B.TO | 64.41%
2y
'RCI.B.TO' RCI.B.TO
downloading symbol: AGU.TO | 62.71%
2y
'AGU.TO' AGU.TO
downloading symbol: TRI.TO | 61.02%
2y
'TRI.TO' TRI.TO
downloading symbol: ABX.TO | 59.32%
2y
'ABX.TO' ABX.TO
downloading symbol: GIB.A.TO | 57.63%
2y
'GIB.A.TO' GIB.A.TO
downloading symbol: L.TO | 55.93%
2y
'L.TO' L.TO
downloading symbol: CVE.TO | 54.24%
2y
'CVE.TO' CVE.TO
downloading symbol: G.TO | 52.54%
2y
'G.TO' G.TO
downloading symbol: PPL.TO | 50.85%
2y
'PPL.TO' PPL.TO
downloading symbol: FTS.TO | 49.15%
2y
'FTS.TO' FTS.TO
downloading symbol: POW.TO | 47.46%
2y
'POW.TO' POW.TO
downloading symbol: QSR.TO | 45.76%
2y
'QSR.TO' QSR.TO
downloading symbol: IMO.TO | 44.07%
2y
'IMO.TO' IMO.TO
downloading symbol: MRU.TO | 42.37%
2y
'MRU.TO' MRU.TO
downloading symbol: FNV.TO | 40.68%
2y
'FNV.TO' FNV.TO
downl

'CBA.AX' CBA.AX
downloading symbol: WBC.AX | 99.00%
2y
'WBC.AX' WBC.AX
downloading symbol: NAB.AX | 98.50%
2y
'NAB.AX' NAB.AX
downloading symbol: ANZ.AX | 98.00%
2y
'ANZ.AX' ANZ.AX
downloading symbol: TLS.AX | 97.50%
2y
'TLS.AX' TLS.AX
downloading symbol: CSL.AX | 97.00%
2y
'CSL.AX' CSL.AX
downloading symbol: BHP.AX | 96.50%
2y
'BHP.AX' BHP.AX
downloading symbol: WES.AX | 96.00%
2y
'WES.AX' WES.AX
downloading symbol: WOW.AX | 95.50%
2y
'WOW.AX' WOW.AX
downloading symbol: MQG.AX | 95.00%
2y
'MQG.AX' MQG.AX
downloading symbol: SCG.AX | 94.50%
2y
'SCG.AX' SCG.AX
downloading symbol: TCL.AX | 94.00%
2y
'TCL.AX' TCL.AX
downloading symbol: WFD.AX | 93.50%
2y
'WFD.AX' WFD.AX
downloading symbol: WPL.AX | 93.00%
2y
'WPL.AX' WPL.AX
downloading symbol: BXB.AX | 92.50%
2y
'BXB.AX' BXB.AX
downloading symbol: RIO.AX | 92.00%
2y
'RIO.AX' RIO.AX
downloading symbol: AMP.AX | 91.50%
2y
'AMP.AX' AMP.AX
downloading symbol: AMC.AX | 91.00%
2y
'AMC.AX' AMC.AX
downloading symbol: SYD.AX | 90.50%
2y
'SYD.AX' S

'TGR.AX' TGR.AX
downloading symbol: WOR.AX | 24.50%
2y
'WOR.AX' WOR.AX
downloading symbol: REG.AX | 24.00%
2y
'REG.AX' REG.AX
downloading symbol: OFX.AX | 23.50%
2y
'OFX.AX' OFX.AX
downloading symbol: JHC.AX | 23.00%
2y
'JHC.AX' JHC.AX
downloading symbol: SFR.AX | 22.50%
2y
'SFR.AX' SFR.AX
downloading symbol: SKC.AX | 22.00%
2y
'SKC.AX' SKC.AX
downloading symbol: GXL.AX | 21.50%
2y
'GXL.AX' GXL.AX
downloading symbol: SXL.AX | 21.00%
2y
'SXL.AX' SXL.AX
downloading symbol: SPK.AX | 20.50%
2y
'SPK.AX' SPK.AX
downloading symbol: MYO.AX | 20.00%
2y
'MYO.AX' MYO.AX
downloading symbol: AHY.AX | 19.50%
2y
'AHY.AX' AHY.AX
downloading symbol: FPH.AX | 19.00%
2y
'FPH.AX' FPH.AX
downloading symbol: GMA.AX | 18.50%
2y
'GMA.AX' GMA.AX
downloading symbol: BRS.AX | 18.00%
2y
'BRS.AX' BRS.AX
downloading symbol: GTY.AX | 17.50%
2y
'GTY.AX' GTY.AX
downloading symbol: NWS.AX | 17.00%
2y
'NWS.AX' NWS.AX
downloading symbol: ISD.AX | 16.50%
2y
'ISD.AX' ISD.AX
downloading symbol: CVO.AX | 16.00%
2y
'CVO.AX' C

## Extra code to run for single ETF symbol's components

In [10]:
def _single_run_download(ETF_symbol, symbols, *args):
    """Get Symbol data"""
    etfi._get_symbol_px_data(ETF_symbol, symbols)

def _after_data_collected(ETF_symbol, symbols, *args):
    """Extract Close Prices"""
    px = etfi.extract_close_prices(symbols)
    ## in case no data returned
    if px is None: raise ValueError('no price data')
    ## must update symbols used in case they are missing in data
    symbols = pd.Series(px.columns)
    
    """Set Key parameters"""
    n_periods = 252
    N_stocks = len(px.columns)
    log_returns = etfi._calc_log_returns(px)
    cuml_returns = etfi._calc_cuml_returns(log_returns)
    start_timestamp = log_returns.ix[-n_periods:].index[0]
    end_timestamp = log_returns.ix[-n_periods:].index[-1]

    """get 52 week highs/lows df's""" 
    highs_list, N_52_highs, highs_df = etfi._52_week_highs(px, symbols)
    lows_list, N_52_lows, lows_df = etfi._52_week_lows(px, symbols)

    """Calculate up/down and below/above stats"""
    avg_pct_below_highs, median_pct_below_highs, avg_pct_above_lows, median_pct_above_lows, \
    N_stocks_off_10, N_stocks_off_20, pct_stocks_off_10, pct_stocks_off_20, \
    N_stocks_up_10, N_stocks_up_20, pct_stocks_up_10, pct_stocks_up_20 = etfi._calc_high_low_metrics(highs_df,
                                                                                                     lows_df,
                                                                                                     N_stocks)

    up, N_up, nup_pct, down, N_down, ndown_pct = etfi._up_down_last_N_periods(n_periods, log_returns)

    """Construct EMAs for dataframe"""
    spans = [252, 63, 21]
    EMAS = etfi._get_EMA(px, spans, symbols)
    n_abv_252, n_abv_63, n_abv_21, pct_abv_252, pct_abv_63, pct_abv_21 = etfi._calc_ema_stats(spans, EMAS, N_stocks)

    """Calculate thresholds"""
    pos_thresh = [0., .01, .02, .03, .04, .05, .06, .07, .1, .15, .2]
    neg_thresh = [ flt * -1. for flt in pos_thresh ]

    N_pos = etfi._calc_return_thresholds(pos_thresh, log_returns.resample('1W', how='sum'), N_stocks, pos=True)
    N_neg = etfi._calc_return_thresholds(neg_thresh, log_returns.resample('1W', how='sum'), N_stocks)
    rev_N_pos = N_pos.copy().iloc[::-1]

    """average return and volatility"""
    avg_cuml_ret, avg_std = etfi._calc_avg_log_returns_vol(log_returns, n_periods)

    """Construct output dataframe for excel"""
    new_odf = etfi._create_output_dataframe(
        n_periods, start_timestamp, end_timestamp, N_stocks, \
        N_52_highs, N_52_lows, \
        avg_pct_below_highs, median_pct_below_highs, avg_pct_above_lows, median_pct_above_lows, \
        N_stocks_off_10, N_stocks_off_20, pct_stocks_off_10, pct_stocks_off_20, \
        N_stocks_up_10, N_stocks_up_20, pct_stocks_up_10, pct_stocks_up_20,
        N_up, nup_pct, N_down, ndown_pct, \
        n_abv_252, n_abv_63, n_abv_21, pct_abv_252, pct_abv_63, pct_abv_21, \
        avg_cuml_ret, avg_std
        )
    time.sleep(1.5)
    """Construct excel format"""
    etfi._create_excelwriter_format(ETF_symbol, new_odf, rev_N_pos, N_neg)    

In [24]:
today = pd.datetime.today().date().strftime('%m-%d-%y')
foreigns = od([('FTSE',FTSE), ('DAX',DAX), ('TSX',TSX), ('ASX',ASX)])

for ETF_symbol, symbols in all_etf_symbols.items():
    _single_run_download(ETF_symbol, symbols)
    _after_data_collected(ETF_symbol, symbols)

downloading symbol: AAPL | 99.80%
downloading symbol: MSFT | 99.60%
downloading symbol: XOM | 99.40%
downloading symbol: JNJ | 99.21%
downloading symbol: GE | 99.01%
downloading symbol: BRK-B | 98.81%
downloading symbol: FB | 98.61%
downloading symbol: T | 98.41%
downloading symbol: PG | 98.21%
downloading symbol: WFC | 98.02%
downloading symbol: JPM | 97.82%
downloading symbol: GOOGL | 97.62%
downloading symbol: GOOG | 97.42%
downloading symbol: AMZN | 97.22%
downloading symbol: VZ | 97.02%
downloading symbol: PFE | 96.83%
downloading symbol: KO | 96.63%
downloading symbol: CVX | 96.43%
downloading symbol: HD | 96.23%
downloading symbol: DIS | 96.03%
downloading symbol: PEP | 95.83%
downloading symbol: MRK | 95.63%
downloading symbol: PM | 95.44%
downloading symbol: CMCSA | 95.24%
downloading symbol: V | 95.04%
downloading symbol: INTC | 94.84%
downloading symbol: CSCO | 94.64%
downloading symbol: BAC | 94.44%
downloading symbol: GILD | 94.25%
downloading symbol: C | 94.05%
downloadin

D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'BRK-B'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'BF-B'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)



downloading symbol: CHRW | 30.36%
downloading symbol: WAT | 30.16%
downloading symbol: COH | 29.96%
downloading symbol: NWL | 29.76%
downloading symbol: WDC | 29.56%
downloading symbol: UHS | 29.37%
downloading symbol: KMX | 29.17%
downloading symbol: KORS | 28.97%
downloading symbol: DOV | 28.77%
downloading symbol: STX | 28.57%
downloading symbol: WU | 28.37%
downloading symbol: AKAM | 28.17%
downloading symbol: EMN | 27.98%
downloading symbol: LLL | 27.78%
downloading symbol: EQT | 27.58%
downloading symbol: MAS | 27.38%
downloading symbol: SRCL | 27.18%
downloading symbol: MLM | 26.98%
downloading symbol: HRS | 26.79%
downloading symbol: KEY | 26.59%
downloading symbol: DGX | 26.39%
downloading symbol: BLL | 26.19%
downloading symbol: TXT | 25.99%
downloading symbol: ARG | 25.79%
downloading symbol: SCG | 25.60%
downloading symbol: CA | 25.40%
downloading symbol: CINF | 25.20%
downloading symbol: LNC | 25.00%
downloading symbol: BBY | 24.80%
downloading symbol: KSU | 24.60%
downlo

D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'AAL.L'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'ABF.L'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)



downloading symbol: ADM.L | 97.03%
downloading symbol: ADN.L | 96.04%

...


downloading symbol: PRG.AX | 1.50%
downloading symbol: TEN.AX | 1.00%

D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'PRG.AX'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'TEN.AX'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)



downloading symbol: WHC.AX | 0.50%
downloading symbol: SGH.AX | 0.00%

D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'WHC.AX'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
D:\python_installs\WinPython-64bit-3.5.1.2\python-3.5.1.amd64\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'SGH.AX'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)



ASX missing_symbols:
Series([], dtype: float64)
